In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
### Define the path to the file (a string in a variable 'db_file')
db_file = '../database/cortona_week.db'

In [3]:
### Créer une connexion vers la base de données
# Attention: si la base de données n'existe pas, elle est créée à l'endroit indiqué.
# Ce qui est une manière de la créer si souhaité
try:
    cn = sql.connect(db_file)
except Exception as e:
    print(e)    
# Le contenu de la variable: connexion
# cn

In [4]:
### La requête à effectuer
# Préparée généralement directement sur la BD SQLite
# Elle produit la liste des années de naissance,
# une année par individu
q1 = """
SELECT 
	T.fk_workshop,
	T.fk_categories, 
	W.fk_event
FROM
	Topics T 
JOIN
	Workshop W 
ON
	T.fk_workshop = W.pk_workshop 
JOIN 
	Categories C 
on
	T.fk_categories = C.pk_categories 
"""
q2 = """
SELECT 
	W.pk_workshop,
    W.name,
    W.fk_event
	
FROM
	Workshop W 

"""
q3 = """
SELECT 
	C.pk_categories,
    C.categories,
    C.meta_category
FROM
	Categories C 
"""

q4 = '''
SELECT
    p.fk_person,
    p.fk_workshop
FROM
    Presentation p

'''

In [5]:
### Création du conteneur du résultat de la requête
cur = cn.cursor()
cur

In [6]:
### Exécuter la requête et récupérer le résultat
cur.execute(q1)
# La méthode 'fetchall' permet de récupérer toutes les lignes
edges_data = cur.fetchall()

cur.execute(q2)
# La méthode 'fetchall' permet de récupérer toutes les lignes
workshop_data = cur.fetchall()

cur.execute(q3)
# La méthode 'fetchall' permet de récupérer toutes les lignes
categories_data = cur.fetchall()

cur.execute(q4)
# La méthode 'fetchall' permet de récupérer toutes les lignes
person_workshop_edge_data = cur.fetchall()


In [7]:
panda_data_edges=pd.DataFrame(edges_data, columns=['workshop_id', 
                                                   'category_id','event_id'])

panda_data_workshop=pd.DataFrame(workshop_data, columns=['workshop_id', 
                                                   'workshop_name','event_id'])
panda_data_categories=pd.DataFrame(categories_data, columns=['category_id', 
                                                   'category_name', 'meta_category'])

panda_data_person_workshop_edge = pd.DataFrame(person_workshop_edge_data, columns =['person_id',
                                                                                   'workshop_id'])



In [8]:
panda_data_categories['category_id']=panda_data_categories['category_id']+15000
panda_data_workshop['workshop_id']=panda_data_workshop['workshop_id']+20000

panda_data_workshop=panda_data_workshop.rename(columns={'workshop_id':'id','workshop_name':'name'})
panda_data_categories=panda_data_categories.rename(columns={'category_id':'id','category_name':'name'})
panda_data_categories['event_id']=0
panda_data_workshop['attribute']='workshop'
panda_data_categories['attribute']='category'
panda_data_workshop['meta_category']=0

panda_nodes_id=pd.concat([panda_data_workshop['id'],panda_data_categories['id']])
panda_nodes_name=pd.concat([panda_data_workshop[['name']],panda_data_categories[['name']]])
panda_nodes_attribute=pd.concat([panda_data_workshop['attribute'],panda_data_categories['attribute']])
panda_nodes_event=pd.concat([panda_data_workshop['event_id'],panda_data_categories['event_id']])
panda_nodes_metacategory=pd.concat([panda_data_workshop['meta_category'],panda_data_categories['meta_category']])

panda_nodes=pd.concat([panda_nodes_id,panda_nodes_name,panda_nodes_attribute,panda_nodes_event, panda_nodes_metacategory],axis=1)
print(panda_nodes)
panda_edges=pd.concat([panda_data_edges['workshop_id']+20000,panda_data_edges['category_id']+15000,panda_data_edges['event_id']],axis=1)
panda_edges=panda_edges.rename(columns={'workshop_id':'source','category_id':'target'})
#panda_edges.describe()
panda_data_person_workshop_edge['workshop_id']=panda_data_person_workshop_edge['workshop_id']+20000
panda_data_person_workshop_edge=panda_data_person_workshop_edge.rename(columns={'workshop_id': 'target',
                                                                               'person_id':'source'})
panda_data_person_workshop_edge['attribute']='person_workshop'
panda_nodes['[z]']=1
panda_nodes.to_csv('nodes_categories.csv',index=False) 
panda_edges.to_csv('edges_categories.csv',index=False) 
panda_data_person_workshop_edge.to_csv('workshop_person_edges.csv',index=False)

        id                                               name attribute  \
0    20011  The Structure of Thought in Western and Easter...  workshop   
1    20012                        Fragmentation and Wholeness  workshop   
2    20013                    The Re-Enchantment of the World  workshop   
3    20014                      Politics, Economy and Science  workshop   
4    20015  Fragmentation and Wholeness in Science and Soc...  workshop   
..     ...                                                ...       ...   
119  15121                                             Europa  category   
120  15122                                Gesellschaftskritik  category   
121  15123                                    Quantummechanik  category   
122  15124                             Künstliche Intelligenz  category   
123  15125                                    Parapsychologie  category   

     event_id  meta_category  
0           1              0  
1           1              0  
2     